# Problem Statement

Each season there are thousands of NCAA basketball games played between Division I men's teams, culminating in March Madness®, the 68-team national championship that starts in the middle of March. We have provided a large amount of historical data about college basketball games and teams, going back many years. Armed with this historical data, you can explore it and develop your own distinctive ways of predicting March Madness® game outcomes. You can even evaluate and compare different approaches by seeing which of them would have done best at predicting tournament games from the past.


## What to Predict
Stage 1 - You should submit predicted probabilities for every possible matchup in the past 4 NCAA® tournaments (2014-2017).

Stage 2 - You should submit predicted probabilities for every possible matchup before the 2018 tournament begins.

In [1]:
import pandas as pd
import numpy as np

# Visualization
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_colwidth', 20)
pd.set_option('expand_frame_repr', False)
pd.options.display.max_rows = 9999

plt.style.use('seaborn-whitegrid')

# this allows plots to appear directly in the notebook
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [3]:
rnd_seed=23
np.random.seed(rnd_seed)

## Data Section 1 - The Basics

**This section provides everything you need to build a simple prediction model and submit predictions.**

- Team ID's and Team Names
- Tournament seeds since 1984-85 season
- Final scores of all regular season, conference tournament, and NCAA® tournament games since 1984-85 season
- Season-level details including dates and region names
- Example submission file for stage 1

### Preview Data

In [4]:
teams = pd.read_csv('./DataFiles/Teams.csv')
seasons = pd.read_csv('./DataFiles/Seasons.csv')
seeds = pd.read_csv('./DataFiles/NCAATourneySeeds.csv')
reg_results = pd.read_csv('./DataFiles/RegularSeasonCompactResults.csv')
tourn_results = pd.read_csv('./DataFiles/NCAATourneyCompactResults.csv')

In [5]:
teams.head()
seasons.head()
seeds.head()
reg_results.head()
tourn_results.head()
ball_data = [teams, seasons, seeds, reg_results, tourn_results]

TeamID     TeamName  FirstD1Season  LastD1Season
0    1101  Abilene Chr           2014          2018
1    1102    Air Force           1985          2018
2    1103        Akron           1985          2018
3    1104      Alabama           1985          2018
4    1105  Alabama A&M           2000          2018

Season     DayZero RegionW    RegionX    RegionY    RegionZ
0    1985  10/29/1984    East       West    Midwest  Southeast
1    1986  10/28/1985    East    Midwest  Southeast       West
2    1987  10/27/1986    East  Southeast    Midwest       West
3    1988   11/2/1987    East    Midwest  Southeast       West
4    1989  10/31/1988    East       West    Midwest  Southeast

Season Seed  TeamID
0    1985  W01    1207
1    1985  W02    1210
2    1985  W03    1228
3    1985  W04    1260
4    1985  W05    1374

Season  DayNum  WTeamID  WScore  LTeamID  LScore WLoc  NumOT
0    1985      20     1228      81     1328      64    N      0
1    1985      25     1106      77     1354      70    H      0
2    1985      25     1112      63     1223      56    H      0
3    1985      25     1165      70     1432      54    H      0
4    1985      25     1192      86     1447      74    H      0

Season  DayNum  WTeamID  WScore  LTeamID  LScore WLoc  NumOT
0    1985     136     1116      63     1234      54    N      0
1    1985     136     1120      59     1345      58    N      0
2    1985     136     1207      68     1250      43    N      0
3    1985     136     1229      58     1425      55    N      0
4    1985     136     1242      49     1325      38    N      0

In [6]:
# Check for null values
info=list(map(lambda x: x.info(), ball_data))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 364 entries, 0 to 363
Data columns (total 4 columns):
TeamID           364 non-null int64
TeamName         364 non-null object
FirstD1Season    364 non-null int64
LastD1Season     364 non-null int64
dtypes: int64(3), object(1)
memory usage: 11.5+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34 entries, 0 to 33
Data columns (total 6 columns):
Season     34 non-null int64
DayZero    34 non-null object
RegionW    34 non-null object
RegionX    34 non-null object
RegionY    34 non-null object
RegionZ    34 non-null object
dtypes: int64(1), object(5)
memory usage: 1.7+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2150 entries, 0 to 2149
Data columns (total 3 columns):
Season    2150 non-null int64
Seed      2150 non-null object
TeamID    2150 non-null int64
dtypes: int64(2), object(1)
memory usage: 50.5+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150684 entries, 0 to 150683
Data columns (total 8 columns):
Season     

In [7]:
# Describe Data
info=list(map(lambda x: x.describe(), ball_data))
for i in info:
    print(i)

            TeamID  FirstD1Season  LastD1Season
count   364.000000     364.000000    364.000000
mean   1282.500000    1988.192308   2017.184066
std     105.221988       7.126666      4.573449
min    1101.000000    1985.000000   1985.000000
25%    1191.750000    1985.000000   2018.000000
50%    1282.500000    1985.000000   2018.000000
75%    1373.250000    1985.000000   2018.000000
max    1464.000000    2014.000000   2018.000000
            Season
count    34.000000
mean   2001.500000
std       9.958246
min    1985.000000
25%    1993.250000
50%    2001.500000
75%    2009.750000
max    2018.000000
            Season       TeamID
count  2150.000000  2150.000000
mean   2001.190233  1291.615814
std       9.563269   103.292788
min    1985.000000  1102.000000
25%    1993.000000  1208.000000
50%    2001.000000  1286.500000
75%    2010.000000  1387.000000
max    2017.000000  1463.000000
              Season         DayNum        WTeamID         WScore        LTeamID         LScore          NumO

Some columns such as TeamID don't make sense to use .describe(), however columns like WScore, LScore could provide us some insight

There are 351 teams currently in Division-I, and an overall total of 364 teams in our team listing (each year, some teams might start being Division-I programs, and others might stop being Division-I programs). 

In [8]:
# how long each team stayed d1 
teams['Length']=teams['LastD1Season']-teams['FirstD1Season']
(teams.sort_values(by='Length', ascending=False)).head()

TeamID      TeamName  FirstD1Season  LastD1Season  Length
182    1283   Missouri St           1985          2018      33
224    1325          Ohio           1985          2018      33
232    1333     Oregon St           1985          2018      33
231    1332        Oregon           1985          2018      33
230    1331  Oral Roberts           1985          2018      33

Division 1 schools are typically the largest universities, and compete in a minimum of 14 sports for both males and females. These schools often have world-class facilities, attract the top athletes in the country, and receive the most media attention.

Division 2 schools are smaller than D1 schools, and student athletes usually finance their education with a combination of athletic and educational scholarships.

Division 3 schools are the smallest of the NCAA institutions. D3 schools are not allowed to offer athletic scholarships.

### Exploratory Data Analysis


In [9]:
# Can conclude most NCAA winners are D1 teams 
teams['Length'].hist(grid=True);

**RegionW, RegionX, Region Y, Region Z** - by convention, the four regions in the final tournament are always named W, X, Y, and Z. Whichever region's name comes first alphabetically, that region will be Region W. And whichever Region plays against Region W in the national semifinals, that will be Region X. For the other two regions, whichever region's name comes first alphabetically, that region will be Region Y, and the other will be Region Z. This allows us to identify the regions and brackets in a standardized way in other files. For instance, during the 2012 tournament, the four regions were East, Midwest, South, and West. Being the first alphabetically, East becomes W. Since the East regional champion (Ohio State) played against the Midwest regional champion (Kansas) in the national semifinals, that makes Midwest be region X. For the other two (South and West), since South comes first alphabetically, that makes South Y and therefore West is Z. So for that season, the W/X/Y/Z are East,Midwest,South,West.

In [10]:
# Seasons
plt.figure(figsize=(10,10))
plt.subplot(221)
sns.countplot(x='RegionW', data=seasons)
plt.xticks(rotation=30)
plt.subplot(222)
sns.countplot(x='RegionX', data=seasons)
plt.xticks(rotation=30)
plt.subplot(223)
sns.countplot(x='RegionY', data=seasons)
plt.xticks(rotation=30)
plt.subplot(224)
s=sns.countplot(x='RegionZ', data=seasons)
plt.xticks(rotation=30);

In [11]:
# merge seasons['TeamId'] with Teams['TeamId'] then merge seed with region
team_history=(seeds.merge(teams,on='TeamID')).merge(seasons,on='Season')
team_history.head()

Season Seed  TeamID        TeamName  FirstD1Season  LastD1Season  Length     DayZero RegionW RegionX  RegionY    RegionZ
0    1985  W01    1207      Georgetown           1985          2018      33  10/29/1984    East    West  Midwest  Southeast
1    1985  W02    1210    Georgia Tech           1985          2018      33  10/29/1984    East    West  Midwest  Southeast
2    1985  W03    1228        Illinois           1985          2018      33  10/29/1984    East    West  Midwest  Southeast
3    1985  W04    1260  Loyola-Chicago           1985          2018      33  10/29/1984    East    West  Midwest  Southeast
4    1985  W05    1374             SMU           1985          2018      33  10/29/1984    East    West  Midwest  Southeast

In [12]:
# convert Region W, X, Y, Z into one column 
team_history.rename(index=str,columns={'RegionW':'W', 'RegionX':'X','RegionY':'Y', 'RegionZ':'Z'}, inplace=True)

In [13]:
# Assign to dummy variable
team_history['Region'] = 'hi'

**Assign one region to each row** 

In [14]:
for i in range(len(team_history)):
    team_history['Region'][i] = team_history[team_history['Seed'][i][0]][i]

/Users/samlin/anaconda/lib/python3.5/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [15]:
# Only need one of the 4 columns, and DayZero just provides us when the season starts
team_history.drop(['W','X','Y','Z','DayZero'],axis=1,inplace=True)

In [16]:
# Assign to dummy variable
team_history['SeedValue'] = 'hi'

**Assign seed value from seed to each row**

In [17]:
for i in range(len(team_history)):
    team_history['SeedValue'][i] = team_history['Seed'][i][1:]

/Users/samlin/anaconda/lib/python3.5/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [18]:
team_history.sample(10)

Season Seed  TeamID     TeamName  FirstD1Season  LastD1Season  Length       Region SeedValue
483     1992  Y10    1222      Houston           1985          2018      33      Midwest        10
1808    2003  W12    1139       Butler           1985          2018      33         East        12
1582    1998  W14    1350     Richmond           1985          2018      33         East        14
210     1988  Y01    1328     Oklahoma           1985          2018      33    Southeast        01
1933    2017  X01    1211      Gonzaga           1985          2018      33         West        01
271     1989  W02    1181         Duke           1985          2018      33         East        02
1529    2005  W15    1151  Chattanooga           1985          2018      33  Albuquerque        15
662     1996  Z01    1345       Purdue           1985          2018      33         West        01
1462    2004  W12    1293    Murray St           1985          2018      33      Atlanta        12
2116    2016  X03    1428         Utah           1985          2018      33      Midwest        03

In [19]:
#what teams had what seed
#what regions had what seed 

In [20]:
team_history.groupby(['TeamName','SeedValue'])['TeamID'].count().reset_index()

TeamName SeedValue  TeamID
0            Air Force        11       1
1            Air Force        13       1
2                Akron        12       1
3                Akron        13       1
4                Akron        15       2
5              Alabama        02       2
6              Alabama        04       1
7              Alabama        05       4
8              Alabama        06       1
9              Alabama        07       2
10             Alabama        08       1
11             Alabama        09       2
12             Alabama        10       2
13         Alabama A&M       16a       1
14          Alabama St        16       2
15          Alabama St       16a       2
16           Albany NY        13       1
17           Albany NY        14       1
18           Albany NY        15       1
19           Albany NY        16       1
20           Albany NY       16a       1
21           Alcorn St        15       1
22           Alcorn St       16a       1
23       American Univ        14       1
24       American Univ        15       2
25      Appalachian St        14       1
26             Arizona        01       6
27             Arizona        02       7
28             Arizona        03       4
29             Arizona        04       2
30             Arizona        05       2
31             Arizona        06       2
32             Arizona        08       2
33             Arizona        09       2
34             Arizona        10       3
35             Arizona        12       1
36          Arizona St        05       1
37          Arizona St        06       1
38          Arizona St        08       1
39          Arizona St        10       2
40     Ark Little Rock        12       1
41     Ark Little Rock        13       1
42     Ark Little Rock        14       1
43     Ark Little Rock        16       1
44     Ark Little Rock       16a       1
45      Ark Pine Bluff       16a       1
46            Arkansas        01       2
47            Arkansas        02       1
48            Arkansas        03       1
49            Arkansas        04       3
50            Arkansas        05       2
51            Arkansas        06       1
52            Arkansas        07       1
53            Arkansas        08       2
54            Arkansas        09       2
55            Arkansas        11       2
56            Arkansas        12       2
57         Arkansas St        15       1
58              Auburn        01       1
59              Auburn        07       1
60              Auburn        08       3
61              Auburn        10       1
62              Auburn        11       1
63         Austin Peay        13       1
64         Austin Peay        14       2
65         Austin Peay        15       1
66         Austin Peay        16       1
67                 BYU        03       1
68                 BYU        04       1
69                 BYU        07       2
70                 BYU        08       4
71                 BYU        10       4
72                 BYU       11a       1
73                 BYU        12       4
74                 BYU       14a       1
75             Ball St        09       1
76             Ball St        11       1
77             Ball St        12       2
78             Ball St        14       1
79             Ball St        15       1
80              Baylor        03       4
81              Baylor        05       1
82              Baylor        06       1
83              Baylor        08       1
84              Baylor        11       1
85             Belmont        11       1
86             Belmont        13       1
87             Belmont        14       1
88             Belmont        15       4
89          Binghamton        15       1
90            Boise St       11a       1
91            Boise St       13a       1
92            Boise St        14       4
93      Boston College        03       1
94      Boston College        04       2
95      Boston College        05       1
96      Boston College        06     

In [21]:
team_history['SeedValue'].unique()

array(['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11',
       '12', '13', '14', '15', '16', '16b', '16a', '11b', '12b', '11a',
       '12a', '14b', '14a', '13b', '13a'], dtype=object)

In [22]:
# Regions with what seed value
sns.set(font_scale=2)
g=sns.factorplot(x="Region",col="SeedValue",data=team_history,kind="count",col_wrap = 5,size=5,aspect=.7);
g.set_xticklabels(rotation='vertical');

In [23]:
team_history.groupby(['Region','SeedValue'])['TeamID'].count().reset_index()

Region SeedValue  TeamID
0       Albuquerque        01       1
1       Albuquerque        02       1
2       Albuquerque        03       1
3       Albuquerque        04       1
4       Albuquerque        05       1
5       Albuquerque        06       1
6       Albuquerque        07       1
7       Albuquerque        08       1
8       Albuquerque        09       1
9       Albuquerque        10       1
10      Albuquerque        11       1
11      Albuquerque        12       1
12      Albuquerque        13       1
13      Albuquerque        14       1
14      Albuquerque        15       1
15      Albuquerque        16       1
16          Atlanta        01       2
17          Atlanta        02       2
18          Atlanta        03       2
19          Atlanta        04       2
20          Atlanta        05       2
21          Atlanta        06       2
22          Atlanta        07       2
23          Atlanta        08       2
24          Atlanta        09       2
25          Atlanta        10       2
26          Atlanta        11       2
27          Atlanta        12       2
28          Atlanta        13       2
29          Atlanta        14       2
30          Atlanta        15       2
31          Atlanta        16       2
32           Austin        01       1
33           Austin        02       1
34           Austin        03       1
35           Austin        04       1
36           Austin        05       1
37           Austin        06       1
38           Austin        07       1
39           Austin        08       1
40           Austin        09       1
41           Austin        10       1
42           Austin        11       1
43           Austin        12       1
44           Austin        13       1
45           Austin        14       1
46           Austin        15       1
47           Austin        16       1
48          Chicago        01       1
49          Chicago        02       1
50          Chicago        03       1
51          Chicago        04       1
52          Chicago        05       1
53          Chicago        06       1
54          Chicago        07       1
55          Chicago        08       1
56          Chicago        09       1
57          Chicago        10       1
58          Chicago        11       1
59          Chicago        12       1
60          Chicago        13       1
61          Chicago        14       1
62          Chicago        15       1
63          Chicago        16       1
64             East        01      30
65             East        02      30
66             East        03      30
67             East        04      30
68             East        05      30
69             East        06      30
70             East        07      30
71             East        08      30
72             East        09      30
73             East        10      30
74             East        11      27
75             East       11a       3
76             East       11b       3
77             East        12      29
78             East       12a       1
79             East       12b       1
80             East        13      30
81             East        14      30
82             East        15      30
83             East        16      24
84             East       16a       6
85             East       16b       6
86   EastRutherford        01       1
87   EastRutherford        02       1
88   EastRutherford        03       1
89   EastRutherford        04       1
90   EastRutherford        05       1
91   EastRutherford        06       1
92   EastRutherford        07       1
93   EastRutherford        08       1
94   EastRutherford        09       1
95   EastRutherford        10       1
96   EastRutherford        11       1
97   EastRutherford        12       1
98   EastRutherford        13       1
99   EastRutherford        14       1
100  EastRutherford        15       1
101  EastRutherford        16       1
102         Midwest        01      29
103         Midwest        02      29
104         Midwest    

In [24]:
# Seeds for which region
g=sns.factorplot(x="SeedValue",col="Region",data=team_history,dodge=True,kind="count",col_wrap = 4,size=5,aspect=1.5)
g.set_xticklabels(rotation='vertical');